# **SIGMA WEDGE CODING CHALLENGE**

*To activate Quantrocket set up the license key*

In [ ]:
from quantrocket.license import set_license
set_license("e7dc7429-dde5-11ee-a0f7-276dcf33ea83")

{'licensekey': 'e7dc........ea83', 'software_license': {}}

In [ ]:
from quantrocket.license import get_license_profile
get_license_profile()

{'licensekey': 'e7dc........ea83', 'software_license': {}}

### **DATA COLLECTION**

*Pulled price data (daily close prices only) for Apple stock (sid='AAPL') for the year 2023 (01-01-2023 to 12-31-2023) from freely available us-stock price data.*

In [ ]:
from quantrocket.master import collect_usstock_listings
collect_usstock_listings()

{'status': 'success', 'msg': 'already up-to-date'}

In [ ]:
from quantrocket.master import get_securities
securities = get_securities(symbols='AAPL', fields=['Sid', 'Symbol', 'Exchange'], vendors='usstock')
securities

,Symbol,Exchange
Sid,,
FIBBG000B9XRY4,AAPL,XNAS


In [ ]:
from quantrocket.zipline import create_usstock_bundle
create_usstock_bundle("aapl-usstock1",free=True)

{'status': 'success', 'msg': 'successfully created aapl-usstock1 bundle'}

In [ ]:
from quantrocket.zipline import ingest_bundle
ingest_bundle("aapl-usstock1")

{'status': 'the data will be ingested asynchronously'}

In [ ]:
from quantrocket import get_prices
data=get_prices('aapl-usstock1', data_frequency='daily', sids="FIBBG000B9XRY4", start_date='2023-01-01', end_date='2023-12-31',fields="Close")

In [ ]:
from quantrocket.zipline import download_bundle_file
download_bundle_file("aapl-usstock1", data_frequency="daily",
                         sids="FIBBG000B9XRY4",
                         start_date="2023-01-01", end_date="2023-12-31",fields="Close",
                         filepath_or_buffer="usstock_aapl.csv")


In [ ]:
import pandas as pd
prices=pd.read_csv('usstock_aapl.csv')
prices

,Field,Date,FIBBG000B9XRY4
0,Close,2023-01-03,124.375
1,Close,2023-01-04,125.658
2,Close,2023-01-05,124.325
3,Close,2023-01-06,128.900
4,Close,2023-01-09,129.427
...,...,...,...
245,Close,2023-12-22,193.600
246,Close,2023-12-26,193.050
247,Close,2023-12-27,193.150
248,Close,2023-12-28,193.580


In [1]:
import pandas as pd
prices=pd.read_csv('/content/file1.csv')

In [2]:
prices.rename(columns = {'FIBBG000B9XRY4':'Stock_price'}, inplace = True)
prices.drop(['Field'],axis=1,inplace=True)
prices

,Date,Stock_price
0,2023-01-03,124.375
1,2023-01-04,125.658
2,2023-01-05,124.325
3,2023-01-06,128.900
4,2023-01-09,129.427
...,...,...
245,2023-12-22,193.600
246,2023-12-26,193.050
247,2023-12-27,193.150
248,2023-12-28,193.580


In [ ]:
data=prices

In [ ]:
prices.columns

Index(['Date', 'Stock_price'], dtype='object')

In [ ]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         250 non-null    object 
 1   Stock_price  250 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.0+ KB


In [ ]:
prices.isnull().sum()

Date           0
Stock_price    0
dtype: int64

### **PRE-SPECIFIED LOGIC**

**Stock Trading Algorithm Overview:**

This Python class, `StockTrading`, represents an algorithm for making trading decisions based on stock price movements. Here's an overview of its functionality:

1. **Initialization:** The class is initialized with the historical stock prices.

2. **Classify States:** Calculate the returns and classify states based on defined thresholds.

3. **Find Portfolio:** Update the portfolio value based on the trading decision.

4. **Update Count:** Update transition counts between states.

5. **Probability Distribution:** Calculate transition probabilities based on transition counts.

6. **Make Decision:** Make trading decisions based on transition probabilities.

7. **Output:** Display the portfolio value, optimal buy indices, and buy dates.

8. **Decision:** Execute the decision-making process for each trading day.


In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

class StockTrading:
    def __init__(self, prices):
        self.prices = prices
        self.state = [0]
        self.portfolio = 0
        self.buy_indices = []
        self.count = {
            -1: {-1: 0, 0: 0, 1: 0},
            0: {-1: 0, 0: 0, 1: 0},
            1: {-1: 0, 0: 0, 1: 0}
        }
        self.transition_probabilities = {-1: {-1: 0, 0: 0, 1: 0}, 0: {-1: 0, 0: 0, 1: 0}, 1: {-1: 0, 0: 0, 1: 0}}

    def classify_states(self):
        returns = [1]
        close_price = list(self.prices['Stock_price'])
        for i in range(1, len(self.prices)):
            r = ((close_price[i] - close_price[i-1]) / close_price[i-1])
            returns.append(r)
            if r >= 0.01:
                self.state.append(1)
            elif r > -0.01:
                self.state.append(0)
            else:
                self.state.append(-1)

    def find_portfolio(self, curr, prev):
        if self.state[curr] == 1 and self.state[prev] == 0:
            self.portfolio += 1
        elif self.state[curr] == -1 and self.state[prev] == 0:
            self.portfolio -= 1

    def update_count(self, prev_state, current_state):
        self.count[prev_state][current_state] += 1

    def probability_distribution(self):
        for prev_state, transitions in self.count.items():
            total_transitions = sum(transitions.values())
            if total_transitions == 0:
                continue
            self.transition_probabilities[prev_state] = {
                next_state: count / total_transitions
                for next_state, count in transitions.items()
            }

        return self.transition_probabilities

    def output(self):
        #transition_distribution = self.probability_distribution()
        index_names = ['Bear', 'Flat', 'Bull']
        column_names = ['Bear', 'Flat', 'Bull']
        matrix = []
        for i in self.transition_probabilities :
            temp = []
            for j in self.transition_probabilities [i]:
                temp.append(self.transition_probabilities [i][j])
            matrix.append(temp)
        """probability_df = pd.DataFrame(matrix, index=index_names, columns=column_names)
        plt.figure(figsize=(8, 6))
        sns.heatmap(probability_df, annot=True, cmap='Blues', fmt=".3f", linewidths=.5)
        plt.title('Probability Distribution')
        plt.xlabel('Next State')
        plt.ylabel('Previous State')
        plt.show()"""

        dates = []
        for i in self.buy_indices:
            dates.append(self.prices.iloc[i]['Date'])
        print("PORTFOLIO VALUE V(N) : ", self.portfolio)
        print("OPTIMAL BUY INDICES : ", self.buy_indices)
        print("BUY DATES : ", dates)
        print("TRANSISTION MATRIX : ")
        for i in matrix:
          print(i)

    def make_decision(self, curr, prev):
        prev_state = self.state[prev]
        current_state = self.state[curr]
        transition_probability = self.probability_distribution()
        #print(transition_probability)

        decision = 0
        additive_state = transition_probability[prev_state].get(1, 0)
        reductive_state = transition_probability[prev_state].get(-1, 0)

        if additive_state > reductive_state:
            decision = 1
            self.buy_indices.append(curr)
            self.find_portfolio(curr, prev)
        elif additive_state < reductive_state:
            decision = 0

        self.update_count(prev_state, current_state)
        return decision, self.portfolio

    def decision(self):
        self.classify_states()
        for i in range(1,len(self.prices)):
            decision, p = self.make_decision(i, i-1)
            #print(f"Day {i+1}: Decision: {decision}, Portfolio Value: {p}")
        self.output()


In [33]:
portfolio = StockTrading(prices)
portfolio.decision()

PORTFOLIO VALUE V(N) :  17
OPTIMAL BUY INDICES :  [5, 6, 8, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 50, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 181, 182, 183, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 203, 204, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 